In [207]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from sklearn.preprocessing import StandardScaler
import numpy as np

In [208]:
df_train = pd.read_csv("train_data.csv")
df_train = df_train.drop(columns="SubwayStation")

df_test = pd.read_csv("test_data.csv")
df_test = df_test.drop(columns=["SubwayStation"])
df_train["SalePrice"].values[0]

141592

Spytac sie prowadzacych czy OHE jest lepsze, czy mapping; jakby ujowo działało, to dorzucic stacje metra jeszcze

In [209]:
def encode_data(df:pd.DataFrame):
    busTimeMapping = {'5min~10min': 2, '0~5min': 1, '10min~15min': 3}
    subwayTimeMapping = {'5min~10min': 2, '0-5min': 1, '10min~15min': 3, '15min~20min':4, 'no_bus_stop_nearby':5}

    df['TimeToBusStop'] = df['TimeToBusStop'].replace(busTimeMapping)
    df['TimeToSubway'] = df['TimeToSubway'].replace(subwayTimeMapping)

    # nie wiem czy jest roznica miedzy mappingiem czy OHE

    df = pd.get_dummies(df, columns=["HallwayType", "HeatingType", "AptManageType"])

    return df



def encode_labels(df:pd.DataFrame):
    i=0
    for value in df.SalePrice.values:
        if value < 100000:
            df.SalePrice.values[i] = 0
        elif value < 350000:
            df.SalePrice.values[i] = 1
        else:
            df.SalePrice.values[i] = 2
        i += 1
    return df

In [210]:
df_train = encode_data(df_train)
df_train = encode_labels(df_train)
df_test = encode_data(df_test)



In [211]:
df_train.SalePrice.value_counts()

SalePrice
1    2998
2     570
0     556
Name: count, dtype: int64

In [212]:
df_train = pd.get_dummies(df_train, columns=["SalePrice"])

In [213]:
train = df_train.sample(frac=0.8, random_state=997)
val = df_train.drop(train.index)

In [214]:
X_train = train.values[:, :-3].astype('float32')
X_val = val.values[:, :-3].astype('float32')
X_test = df_test.values.astype('float32')

Y_train = train.values[:, -3:].astype('float32')
Y_val = val.values[:, -3:].astype('float32')

In [215]:
# normalizacja scalerem filtrowanym do danych treningowych
scaler = StandardScaler()
scaler.fit(np.vstack((X_train, X_val)))
X_train_norm = scaler.transform(X_train)
X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_test)


Albo zpreparować dane w odpowiedni sposób (podział na klasy), albo zroibić funkcję kloca, ktora zrzutuje labele i predy na klasy

In [230]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

device

device(type='cuda')

In [217]:
train_ds = data.TensorDataset(torch.from_numpy(X_train_norm).to(device),torch.from_numpy(Y_train).to(device))
val_ds = data.TensorDataset(torch.from_numpy(X_val_norm).to(device),torch.from_numpy(Y_val).to(device))
test_ds = data.TensorDataset(torch.from_numpy(X_test_norm).to(device),torch.zeros(X_test_norm.shape[0], 1).to(device))

train_loader = data.DataLoader(train_ds, shuffle=True, batch_size=500)
val_loader = data.DataLoader(val_ds)
test_loader = data.DataLoader(test_ds, batch_size=500)

In [218]:
class Aproximator(nn.Module):

    def __init__(self, num_inputs, num_labels, hidden_size = [30, 30, 30], BN=False, dropout=0):
        super(Aproximator,self).__init__()

        layers = []
        layer_sizes = [num_inputs] + hidden_size
        for layer in range (1, len(layer_sizes)):
            if BN:
                layers += [nn.Linear(layer_sizes[layer-1], layer_sizes[layer]),
                           nn.BatchNorm1d(layer_sizes[layer]),
                           nn.Dropout(dropout),
                           torch.nn.Tanh()
                           ]
            else:
                layers += [nn.Linear(layer_sizes[layer-1], layer_sizes[layer]),
                           nn.Dropout(dropout),
                           torch.nn.Tanh()
                           ]
        layers += [nn.Linear(layer_sizes[-1], num_labels)]

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)


In [237]:
####### Parametry
weight_decay = 0
lr = 1e-3


#######
Aprox1 = Aproximator(X_train_norm.shape[1], 3, [50, 30, 15]).to(device)


optim = torch.optim.Adam(Aprox1.parameters(), weight_decay=weight_decay, lr=lr)
loss_fnc = torch.nn.CrossEntropyLoss(weight=torch.tensor([1, 5, 5]))
next(Aprox1.parameters()).device

device(type='cuda', index=0)

In [235]:
def train(data_loader, epoch):

    Aprox1.train(mode=True)

    for n in range(epoch):

        epoch_losses = []
        for input, label in data_loader:

            pred = Aprox1.forward(input)
            print(pred.device)
            print(label.device)
            loss = loss_fnc(pred, label)

            optim.zero_grad()
            loss.backward()
            optim.step()

            epoch_losses.append(loss.item())

        loss_mean = np.array(epoch_losses).mean()

        print(f"Epoch {n} loss {loss_mean:.3}")

    Aprox1.eval()


In [236]:
train(train_loader, 50)


cuda:0
cuda:0


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!